In [1]:
from google.cloud import bigquery
from google.cloud import pubsub
import pandas_gbq
import json
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
publish_client = pubsub.PublisherClient()

In [3]:
topic = 'projects/crack-descent-299314/topics/chicago-topic'
sql = """
SELECT * FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` LIMIT 100000
"""
df = pandas_gbq.read_gbq(sql, project_id=os.environ.get("PROJECT_ID"))
df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])

Downloading: 100%|██████████| 100000/100000 [00:34<00:00, 2908.99rows/s]


In [4]:
load = df.to_json(orient='records')
load = json.loads(load)

In [23]:
for x in load:
    x['trip_start_timestamp'] = int(x['trip_start_timestamp']/10000)
    x['trip_end_timestamp'] = int(0 if x['trip_end_timestamp'] is None else x['trip_end_timestamp']/1000)
    feature = publish_client.publish(topic,json.dumps(x).encode('utf-8'))
    #print(json.dumps(x).encode('utf-8'))
    #print(feature.result())
print('done')

done
